In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install fastapi > /dev/null
!pip install uvicorn > /dev/null
!pip install pydantic > /dev/null
!pip install scikit-learn > /dev/null
!pip install requests > /dev/null
!pip install pypi-json > /dev/null
!pip install pyngrok > /dev/null
!pip install nest-asyncio v > /dev/null

!pip install pymongo > /dev/null
!pip install pydub > /dev/null

In [ ]:
!pip install pydub > /dev/null

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

In [ ]:
from ConvertToWav import convert_mp4_to_wav
from DatabaseConnection import retrieve_mp4_from_gridfs, insert_transcript
from Transcript import get_embedding_model, get_whisper_model, generate_speaker_labeled_transcript

In [ ]:
from fastapi import FastAPI
from fastapi.responses import Response
from pydantic import BaseModel
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import asyncio

In [ ]:
embedding_model = get_embedding_model()

l_model = get_whisper_model()

In [ ]:
app = FastAPI()

# Set the allowed origins for CORS
origins = ["*"]

# Add CORS middleware to the app
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Define the model input schema
class model_input(BaseModel):
    Path: str
    Num: str

# Define the endpoint for the '/api/transcript' route
@app.post('/api/transcript')
async def test(input_dictionary: model_input):

    # Extract the path and num from the input dictionary
    path_of_gridfs = input_dictionary.Path
    num = input_dictionary.Num

    # Retrieve the MP4 file from GridFS
    path_of_mp4 = await asyncio.to_thread(retrieve_mp4_from_gridfs, path_of_gridfs)

    # Convert the MP4 file to WAV
    path_of_wav = await asyncio.to_thread(convert_mp4_to_wav, path_of_mp4)

    # Generate the speaker-labeled transcript using the WAV file and num parameter
    transcript = await asyncio.to_thread(generate_speaker_labeled_transcript, path_of_wav, int(num), l_model, embedding_model)

    # Insert the transcript into the database
    await asyncio.to_thread(insert_transcript, path_of_gridfs, transcript)

    # Return a successful response
    return Response(status_code=200)


In [ ]:
# Set ngrok authentication token
ngrok.set_auth_token('2RnPFwiy2ix2kateedVcbcVMbFx_7WjwaAX2NTud7b924XnmW')

# Establish ngrok tunnel to expose local server
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL generated by ngrok
print('Public URL:', ngrok_tunnel.public_url)

# Enable compatibility between asyncio and uvicorn event loop
nest_asyncio.apply()

# Run the FastAPI application using uvicorn server on port 8000
uvicorn.run(app, port=8000)
